In [1]:

#!pip install openpyxl


In [2]:
import pandas as pd
import glob
import os
import openpyxl
import altair as alt
# use the 538 theme
alt.themes.enable('fivethirtyeight')

ThemeRegistry.enable('fivethirtyeight')

In [3]:
import pandas as pd

all_sheets = pd.read_excel("Historical_Expenditures.xlsx", sheet_name=None)
print(all_sheets.keys())

df = all_sheets['Pivot Table Data']

df['Amount'] *= 1000

df = df.drop(columns=[col for col in df.columns if 'Unnamed:' in col])
# Drop rows with missing 'Amount' data
df = df.dropna(subset=['Amount'])

# Convert 'Fiscal Year' to datetime
df['Fiscal Year'] = df['Fiscal Year'].apply(lambda x: pd.to_datetime(x.split('-')[0]))


df['Year'] = df['Fiscal Year'].dt.year


#Exclude unofficial 2023 budget
df = df[(df['Year'] <= 2022)]


# Exclude Federal Funds
df_non_federal = df[df['Fund'] != 'Federal Funds']

# Group by 'Fiscal Year' and 'Department' and calculate the total 'Amount'
#exp_grouped = expenditures.groupby(['Fiscal Year', 'Source','Year'])['Amount'].sum().reset_index()


# Define the old strings to replace and the new string
old_strings = ['Department of Health Services', 'State Department of Health Care Services']
new_string = 'Department of Health Care Services'

old_strings2 = ['Department of Corrections', 'Department of Corrections & Rehabilitation']
new_string2 = 'Corrections and Rehabilitation'


# Replace the old strings with the new string
df_non_federal = df_non_federal.replace(old_strings, new_string)
df_non_federal = df_non_federal.replace(old_strings2, new_string2)

df_non_federal.head()



#data = alt.Data(values=rev_grouped.to_dict(orient='records'))



dict_keys(['Pivot Table', 'Pivot Table Data'])


,Dept. Code,Department,Function,Fiscal Year,Fund,DOF Agency,Debt Service?,Amount,Year
0,1110,Department of Consumer Affairs - Regulatory Bo...,State Operations,1984-01-01,General Fund,"Business, Consumer Services, and Housing",Not GO Debt Service,1817000.0,1984
1,1110,Department of Consumer Affairs - Regulatory Bo...,State Operations,1984-01-01,Special Funds,"Business, Consumer Services, and Housing",Not GO Debt Service,92824000.0,1984
4,1110,Department of Consumer Affairs - Regulatory Bo...,State Operations,1985-01-01,General Fund,"Business, Consumer Services, and Housing",Not GO Debt Service,1966000.0,1985
5,1110,Department of Consumer Affairs - Regulatory Bo...,State Operations,1985-01-01,Special Funds,"Business, Consumer Services, and Housing",Not GO Debt Service,103422000.0,1985
8,1110,Department of Consumer Affairs - Regulatory Bo...,State Operations,1986-01-01,General Fund,"Business, Consumer Services, and Housing",Not GO Debt Service,1960000.0,1986


In [4]:

agency_grouped = df_non_federal.groupby(['Fiscal Year', 'DOF Agency', 'Year'])['Amount'].sum().reset_index()

agency_grouped['yaxis'] = agency_grouped['Amount']/1000000000


agencydata = alt.Data(values=agency_grouped.to_dict(orient='records'))

# Create a stacked area chart
agencychart = alt.Chart(agencydata).mark_area().encode(
    x=alt.X('Fiscal Year:T', title = 'Year',  axis=alt.Axis(grid=False)),
    y=alt.Y('yaxis:Q', title = 'Billions of Dollars', stack='zero'),  # Use 'stack="zero"' for non-normalized
    color=alt.Color('DOF Agency:N', legend = None),
    tooltip=[
            alt.Tooltip('DOF Agency:N', title='State Agency'),
            alt.Tooltip('Year:O', title='Year'),
            alt.Tooltip('Amount:Q', title='Amount', format='$,.2f')
        ]
).properties(
        title = alt.TitleParams(
            text= ("Total State Expenditures Over Time by State Agency"),
            subtitle= ("Total expenditure data not available before 1985",
                        "Hover over any area on the stacked area chart to see its info"),
            fontSize = 20,
            subtitleFontSize= 15),
    width=600,
    height=500,)

agencychart


alt.Chart(...)

In [5]:
agencychart.save("agencychart.json") 

Legislative Analyst's Office: https://lao.ca.gov/policyareas/state-budget/historical-data

In [6]:
dept_grouped = df_non_federal.groupby(['Department', 'Year'])['Amount'].sum().reset_index()


# Sort the departments by 2022 spending and take the top 20
top_departments = (
    dept_grouped[dept_grouped['Year'] == 2022]
    .groupby('Department')
    .sum()
    .sort_values('Amount', ascending=False)
    .head(40)
    .index
    .tolist()
)

# Filter data_long to include only the top 40 departments from 2022
dept_filtered = dept_grouped[dept_grouped['Department'].isin(top_departments)]


# Create the chart
chart_2017 = (
    alt.Chart(dept_filtered[dept_filtered['Year'] == 2017])
    .mark_bar()
    .encode(
        x=alt.X('Amount:Q', title='US Dollar Amount'),
        y=alt.Y(
            'Department:N',
            title='Department',
            sort=top_departments
        ),
        color=alt.Color('Year:N'),
        opacity=alt.value(0.3),
        tooltip=[
            alt.Tooltip('Department', title='Department'),
            alt.Tooltip('Year', title='Year'),
            alt.Tooltip('Amount', title='Amount', format='$,.2f')
        ]
    )
)

chart_2022 = (
    alt.Chart(dept_filtered[dept_filtered['Year'] == 2022])
    .mark_bar()
    .encode(
        x=alt.X('Amount:Q', title='US Dollar Amount'),
        y=alt.Y(
            'Department:N',
            title='Department',
            sort=top_departments
        ),
        color='Year:N',
        opacity=alt.value(0.5),
        tooltip=[
            alt.Tooltip('Department', title='Department'),
            alt.Tooltip('Year', title='Year'),
            alt.Tooltip('Amount', title='Amount', format='$,.2f')
        ]
    )
)

layeredexpenditures = alt.layer(chart_2022, chart_2017).properties(
        title = alt.TitleParams(
            text= ("Layered Bar Chart Comparing 2017 and 2022 Expenditures by Department"),
            subtitle= ("2017 and 2022 expenditures compared, sorted by top 40 departments with highest 2022 Funding"),
            fontSize = 20,
            subtitleFontSize= 15))

layeredexpenditures

alt.LayerChart(...)

In [7]:
layeredexpenditures.save("layeredexpenditures.json") 

In [8]:

dept_grouped = df_non_federal.groupby(['Fiscal Year', 'Department', 'Year'])['Amount'].sum().reset_index()

dept_grouped['yaxis'] = dept_grouped['Amount']/1000000000



# Filter data_long to include only the top 40 departments from 2022
dept_filtered = dept_grouped[dept_grouped['Department'].isin(top_departments)]

dept_data = alt.Data(values=dept_filtered.to_dict(orient='records'))


# Line Chart
line = alt.Chart(dept_data).mark_line(point=True).encode(
    x=alt.X('Fiscal Year:T', title = 'Year',  axis=alt.Axis(grid=False)),
    y=alt.Y('yaxis:Q', title='Billions of Dollars',  axis=alt.Axis(grid=False), scale=alt.Scale(domain=(0, 75))),
    color=alt.Color('Department:N', legend = None),
    tooltip=[
            alt.Tooltip('Department:N', title='State Department'),
            alt.Tooltip('Year:O', title='Year'),
            alt.Tooltip('Amount:Q', title='Amount', format='$,.2f')
        ]
)

#make a selection that will highlight individual lines based on the country name for the point that the mouse is nearest to   
highlight = alt.selection(type='single', on='mouseover',
                      fields=['Department'], nearest=True)

#mark the invisible points adding the highlight selection that determines which field to highlight based on mouseover proximity
points1 = line.mark_circle().encode(
opacity=alt.value(0)
).add_selection(
highlight
)

#line chart that increases the size of the lines of the base line chart based on the condition of whether the mouse is closest
#to a point on the highlight selection being used on the points
lines = line.mark_line().encode(
size=alt.condition(highlight, alt.value(2), alt.value(.3))
)


#layer the points and lines charts and add the dropdown selection that filters the data accordingly
totalexpenditures = alt.layer(
    points1, lines).properties(
        title = alt.TitleParams(
            text= ("Total State Expenditures Over Time by State Department"),
            subtitle= ("There are multiple departments within each state agency",
                       "Hover over any point on a line to see its info and highlight the line"),
            fontSize = 20,
            subtitleFontSize= 15),
    width=600,
    height=500,)

totalexpenditures

# .properties(
#     width=700,
#     height=400,
#     title='Total Expenditures Over Time by State Agency (Excluding Federal Funds)'
# )
# line_chart.interactive()



alt.LayerChart(...)

In [9]:
totalexpenditures.save("totalexpenditures.json") 

In [10]:



import pandas as pd

all_sheets = pd.read_excel("Historical_Revenues (1).xlsx", sheet_name=None)
revenues = all_sheets['Pivot Table Raw Data'] 
revenues['Amount'] *= 1000

revenues = revenues.drop(columns=[col for col in df.columns if 'Unnamed:' in col])
# Drop rows with missing 'Amount' data
revenues = revenues.dropna(subset=['Amount'])

# Convert 'Fiscal Year' to datetime
revenues['Fiscal Year'] = revenues['Fiscal Year'].apply(lambda x: pd.to_datetime(x.split('-')[0]))

#revenues['Year'] = pd.to_datetime(revenues['Fiscal Year'], format='%Y')

revenues['Year'] = revenues['Fiscal Year'].dt.year


totalrevenues = revenues[(revenues['Year'] <= 2017)]





In [11]:
# Group by 'Fiscal Year' and 'Department' and calculate the total 'Amount'
rev_grouped = totalrevenues.groupby(['Fiscal Year', 'Source','Year'])['Amount'].sum().reset_index()

rev_grouped['yaxis'] = rev_grouped['Amount']/1000000000

data2 = alt.Data(values=rev_grouped.to_dict(orient='records'))


revenueschart = alt.Chart(data2).mark_area().encode(
    x=alt.X('Fiscal Year:T', axis=alt.Axis(grid=False)),
    y=alt.Y('yaxis:Q', title="Billions of Dollars", stack='zero'),  # Use 'stack="zero"' for non-normalized
    color=alt.Color('Source:N', legend = None),
    tooltip=[alt.Tooltip('Source:N', title='Source'),
            alt.Tooltip('Year:O', title='Year'),  # Changed 'Fiscal Year:T' to 'Year:O'
            alt.Tooltip('Amount:Q', title='Amount', format='$,.2f')]
).properties(
        title = alt.TitleParams(
            text= ("Total State Revenues Over Time By Source"),
            subtitle= ("Total revenue data not available beyond 2017",
                       "Hover over any point on a line to see its info and highlight the line"),
            fontSize = 20,
            subtitleFontSize= 15),
    width=600,
    height=500,)

revenueschart

alt.Chart(...)

In [12]:
revenueschart.save("revenueschart.json") 

In [13]:
# Calculate total revenues per fiscal year
revenues_per_year = revenues.groupby(['Fiscal Year'])['Amount'].sum().reset_index()
revenues_per_year = revenues_per_year[(revenues_per_year['Fiscal Year'].dt.year <= 2017)]

# Group by 'Fiscal Year' and 'Department' and calculate the total 'Amount'
expenditures_per_year = df_non_federal.groupby(['Fiscal Year'])['Amount'].sum().reset_index()

GDPdata = pd.read_excel("Table (1).xlsx")


#data = alt.Data(values=df_grouped.to_dict(orient='records'))

GDPdata = GDPdata.T
                      
GDPdata.columns = GDPdata.iloc[0]
GDPdata = GDPdata[1:].reset_index()
GDPdata['Fiscal Year'] = GDPdata['index'].apply(lambda x: pd.to_datetime(x.split('-')[0]))
GDPdata = GDPdata[['Fiscal Year','GDP 2022 Dollars','GDP Chained 2012 Dollars']]

result = GDPdata.merge(expenditures_per_year, on='Fiscal Year', how='outer').merge(revenues_per_year, on='Fiscal Year', how='outer')

result = result.rename(columns={'Amount_x': 'Total CA Expenditures', 'Amount_y': 'Total CA Revenues'})

result['GDP 2022 Dollars'] *= 1000000
result['GDP Chained 2012 Dollars'] *= 1000000

result

,Fiscal Year,GDP 2022 Dollars,GDP Chained 2012 Dollars,Total CA Expenditures,Total CA Revenues
0,1998-01-01,1147520400000.0,1468730800000.0,7.525288e+10,7.428089e+10
1,1999-01-01,1241899700000.0,1574305900000.0,8.485873e+10,8.753573e+10
2,2000-01-01,1356975400000.0,1696172400000.0,9.637914e+10,8.841904e+10
3,2001-01-01,1375761300000.0,1692324100000.0,9.921956e+10,8.980409e+10
4,2002-01-01,1418429600000.0,1722522400000.0,1.067780e+11,9.579430e+10
...,...,...,...,...,...
68,1979-01-01,NaN,NaN,NaN,2.091882e+10
69,1980-01-01,NaN,NaN,NaN,2.210379e+10
70,1981-01-01,NaN,NaN,NaN,2.360088e+10
71,1982-01-01,NaN,NaN,NaN,2.429106e+10


In [14]:


# Melt the data into a long format
melted_df = result.melt('Fiscal Year', var_name='Measurement', value_name='Dollars')

melted_df['yaxis'] = melted_df['Dollars']/1000000000

# Create a line chart
REGDPchart = alt.Chart(melted_df).mark_line().encode(
    x=alt.X('Fiscal Year:T', title = 'Year', axis=alt.Axis(grid=False)),
    y=alt.Y('yaxis:Q', title = "Billions of Dollars", scale=alt.Scale(domain=(0, 3800))),
    color=alt.Color('Measurement:N', scale=alt.Scale(
        domain=['GDP 2022 Dollars', 'GDP Chained 2012 Dollars', 'Total CA Expenditures', 'Total CA Revenues'],
        range=['purple', 'orange', 'red', 'green'])),
    tooltip=[alt.Tooltip('Measurement', title='Measurement'),
            alt.Tooltip('Fiscal Year', title='Year'),
            alt.Tooltip('Dollars', title='Dollars', format='$,.2f')]
).properties(
        title = alt.TitleParams(
            text= ("California Total Revenues and Expenditures Compared to the State's GDP"),
            subtitle= ("The exponential growth from the previous chart suddenly looks very modest"),
            fontSize = 20,
            subtitleFontSize= 15),
    width=600,
    height=500,)

REGDPchart



alt.Chart(...)

In [15]:
REGDPchart.save("Revenues-Expenditures-GDP.json") 

Source: U.S. Bureau of Economic Analysis  Release: Gross Domestic Product by State  


In [16]:
nogdp = melted_df[(melted_df['Measurement'] != 'GDP 2022 Dollars') & (melted_df['Measurement'] != 'GDP Chained 2012 Dollars')]



# Create a line chart
REchart = alt.Chart(nogdp).mark_line().encode(
    x=alt.X('Fiscal Year:T',  title = 'Year', axis=alt.Axis(grid=False)),
    y=alt.Y('yaxis:Q', title = 'Billions of Dollars'),
    color=alt.Color('Measurement:N', scale=alt.Scale(
        domain=['Total CA Expenditures', 'Total CA Revenues'],
        range=['red', 'green']))
).properties(
        title = alt.TitleParams(
            text= ("Total Revenues (1950-2017) and Total Expenditures (1985-2022)"),
            subtitle= ("Neither dataset is complete, but spending has generally tracked with revenues and shows exponential growth"),
            fontSize = 20,
            subtitleFontSize= 15),
    width=600,
    height=500,)

REchart



alt.Chart(...)

In [17]:
REchart.save("Revenues-Expenditures.json") 

In [18]:
# Importing necessary libraries
import pandas as pd

# Loading the data
data = pd.read_csv('60000_1997_2022.csv')

# Displaying the first few rows of the data
#data.head()

In [19]:
# Removing rows with NaN values
data = data.dropna()



# Removing the '$' sign and converting the GDP values to floats
for year in range(1997, 2022):
    data[str(year)] = data[str(year)].replace('[\$,]', '', regex=True).astype(float)

# Melting the DataFrame to get 'Year', 'Industry' and 'GDP' columns
data_melted = data.melt(id_vars=['Level1', 'Level2', 'Level3'], var_name='Year', value_name='GDP')

# Renaming the 'Level3' column to 'Industry'
data_melted = data_melted.rename(columns={'Level3': 'Industry'})

data_melted['GDP'] *= 1000000

# Displaying the first few rows of the preprocessed data
data_melted.head()

,Level1,Level2,Industry,Year,GDP
0,Private industries,"Agriculture, forestry, fishing...",Farms,1997,1.885300e+10
1,Private industries,"Agriculture, forestry, fishing...","Forestry, fishing, and...",1997,4.409900e+09
2,Private industries,"Mining, quarrying, and oil and...",Oil and gas extraction,1997,1.789040e+10
3,Private industries,"Mining, quarrying, and oil and...",Mining (except oil and...,1997,3.203500e+09
4,Private industries,"Mining, quarrying, and oil and...",Support activities for...,1997,7.538000e+08


In [20]:
# Importing necessary library
data_melted['yaxis'] = data_melted['GDP']/1000000000
data_melted['Year2'] = data_melted['Year']

# Creating the stacked area chart
gdpchart = alt.Chart(data_melted).mark_area().encode(
    x=alt.X('Year:T', title='Year', axis=alt.Axis(grid=False)),
    y=alt.Y('yaxis:Q', title='Billions of Chained 2012 Dollars', stack='zero', axis=alt.Axis(grid=True)),
    color=alt.Color('Industry:N', title='Industry', legend=None),
    tooltip=[alt.Tooltip('Industry', title='Industry'),
            alt.Tooltip('Year2:O', title='Year'),
            alt.Tooltip('GDP', title='GDP', format='$,.2f')]
    
).properties(
        title = alt.TitleParams(
            text= ("Stacked Area Chart of California GDP by Industry (1997 - 2022)"),
            subtitle= ("Hover over any area on the stacked area chart to see the industry specific dollar amount per year"),
            fontSize = 20,
            subtitleFontSize= 15),
    width=600,
    height=500)

gdpchart

alt.Chart(...)

In [21]:
gdpchart.save("GDP-stacked.json") 

data from https://united-states.reaproject.org/data-tables/gsp-a900n/tools/60000/

In [22]:
data_melted['yaxis'] = data_melted['GDP']/1000000000


# Line Chart
line = alt.Chart(data_melted).mark_line(point=True).encode(
    x=alt.X('Year:T', title = 'Year', axis=alt.Axis(grid=False)),
    y=alt.Y('yaxis:Q', title='Billions of Chained 2012 Dollars', axis=alt.Axis(grid=False)),
    color=alt.Color('Industry:N', legend = None),
    tooltip=[
            alt.Tooltip('Industry:N', title='Industry'),
            alt.Tooltip('Year2:O', title='Year'),
            alt.Tooltip('GDP:Q', title='GDP', format='$,.2f')
        ]
)

#make a selection that will highlight individual lines based on the country name for the point that the mouse is nearest to   
highlight = alt.selection(type='single', on='mouseover',
                      fields=['Industry'], nearest=True)

#mark the invisible points adding the highlight selection that determines which field to highlight based on mouseover proximity
points1 = line.mark_circle().encode(
opacity=alt.value(0)
).add_selection(
highlight
)

#line chart that increases the size of the lines of the base line chart based on the condition of whether the mouse is closest
#to a point on the highlight selection being used on the points
lines = line.mark_line().encode(
size=alt.condition(highlight, alt.value(2), alt.value(.3))
)


#layer the points and lines charts and add the dropdown selection that filters the data accordingly
GPDindustry = alt.layer(
    points1, lines).properties(
        title = alt.TitleParams(
            text= ("Line Chart of California GDP by Industry (1997 - 2022)"),
            subtitle= ("Hover over any point on a line to see its info and highlight the line"),
            fontSize = 20,
            subtitleFontSize= 15),
    width=600,
    height=500,)

GPDindustry


alt.LayerChart(...)

In [23]:
GPDindustry.save("GDP-line.json") 